## Preprocessing for ENSANUT 2012 - Nutricion + Salud

https://ensanut.insp.mx/encuestas/ensanut2012/descargas.php

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
import os


In [2]:
# Leer los datasets
nutricion = pd.read_csv("../data/processed/completo_final_nutricion.csv")
salud = pd.read_csv("../data/processed/completo_final_salud.csv")

# Revisar shapes
print("Nutricion:", nutricion.shape)
print("Salud:", salud.shape)

Nutricion: (27163, 117)
Salud: (28202, 21)


In [3]:
nutricion.columns.tolist()

['folio',
 'intp',
 'entidad',
 'municipio',
 'localidad',
 'sexo',
 'edad_meses',
 'edad_anios',
 'peso_kg',
 'talla_cm',
 'zscore_imc_para_edad',
 'clasificacion_imc',
 'ponderador',
 'region',
 'id_persona',
 'energia_kcal',
 'proteina_g',
 'grasas_totales_g',
 'carbohidratos_totales_g',
 'fibra_total_g',
 'azucares_totales_g',
 'calcio_mg',
 'hierro_mg',
 'zinc_mg',
 'vitamina_c_mg',
 'tiamina_mg',
 'riboflavina_mg',
 'niacina_mg',
 'acido_pantotenico_mg',
 'energia_kcal_nutri',
 'proteina_g_nutri',
 'grasas_totales_g_nutri',
 'carbohidratos_totales_g_nutri',
 'fibra_total_g_nutri',
 'azucares_totales_g_nutri',
 'calcio_mg_nutri',
 'hierro_mg_nutri',
 'zinc_mg_nutri',
 'vitamina_c_mg_nutri',
 'tiamina_mg_nutri',
 'riboflavina_mg_nutri',
 'niacina_mg_nutri',
 'acido_pantotenico_mg_nutri',
 'folato_total_ug',
 'vitamina_b12_ug',
 'vitamina_a_rae_ug',
 'vitamina_d_mcg',
 'vitamina_e_mg',
 'vitamina_k_ug',
 'num_integrante',
 'edad_dias',
 'fecha_nacimiento',
 'fecha_visita',
 'altitud

In [4]:
salud.columns.tolist()

['folio_consecutivo',
 'intp',
 'entidad',
 'munici',
 'locali',
 'sexo',
 'edad',
 'dia_nac',
 'mes_nac',
 'anio_nac',
 'est_urb',
 'est_marg',
 'pondef',
 'id_persona',
 'folio_vivienda',
 'meses',
 'afilia_1ra',
 'afilia_tras',
 'pondei',
 'deciles',
 'pondeh']

In [6]:
# ==============================================================================
# Merge final para consolidar el dataset nutrición + salud
# ==============================================================================
# Objetivo del merge:
# - Construir un dataset final para análisis del crecimiento infantil
#   que integre información nutricional (talla, peso, IMC, nutrientes) con
#   datos adicionales de salud (hemoglobina, anemia, afiliación, estratos).

# Razón para usar LEFT JOIN:
# - El dataset base es nutrición, ya que contiene las variables principales de crecimiento.
# - Los datos de salud aportan información complementaria (anemia), por lo que
#   no queremos perder registros nutricionales si no tienen datos de salud.
# - Por eso usamos un LEFT JOIN, asegurando que el dataset final tenga **todos los niños
#   con datos nutricionales**, agregando columnas de salud sólo donde exista información.

# ==============================================================================
# Preparar columnas para evitar duplicados
# ==============================================================================
# Dejar solo columnas únicas de salud que no estén ya en nutrición, salvo id_persona
columnas_salud = [
    col for col in salud.columns 
    if col not in nutricion.columns or col == "id_persona"
]
salud_reducido = salud[columnas_salud]

# ==============================================================================
# Merge final por 'id_persona'
# ==============================================================================
completo_final = nutricion.merge(
    salud_reducido,
    on='id_persona',
    how='left'
)

# Revisar el resultado
print("Shape del dataset final:", completo_final.shape)
print("IDs únicos:", completo_final["id_persona"].nunique())

# ==============================================================================
# Guardar el dataset final consolidado
# ==============================================================================
output_path = "../data/processed/completo_final_nutricion_salud.csv"
completo_final.to_csv(output_path, index=False)
print(f"Archivo guardado en: '{output_path}'")


Shape del dataset final: (27163, 134)
IDs únicos: 27163
Archivo guardado en: '../data/processed/completo_final_nutricion_salud.csv'


In [7]:
completo_final

,folio,intp,entidad,municipio,localidad,sexo,edad_meses,edad_anios,peso_kg,talla_cm,...,est_urb,est_marg,pondef,folio_vivienda,meses,afilia_1ra,afilia_tras,pondei,deciles,pondeh
0,100001.0,3.0,10.0,1.0,5.0,2.0,104.344969,96107.0,27.150,132.45,...,1.0,2.0,258.621644,100001,7.0,0.0,0.0,137.820655,3.0,148.069240
1,100006.0,6.0,10.0,1.0,5.0,2.0,134.537988,132143.0,32.075,140.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100008.0,10.0,10.0,1.0,5.0,1.0,90.743326,8495.0,21.200,118.40,...,1.0,2.0,353.637540,100008,5.0,0.0,0.0,153.748303,1.0,148.069240
3,100009.0,3.0,10.0,1.0,5.0,2.0,75.334702,7283.0,32.475,119.95,...,1.0,2.0,143.968925,100009,2.0,0.0,0.0,156.810281,2.0,148.069240
4,100010.0,7.0,10.0,1.0,5.0,1.0,97.741273,96107.0,21.300,132.10,...,1.0,2.0,353.293010,100010,0.0,6.0,6.0,171.843150,1.0,148.069240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27158,92321.0,7.0,9.0,17.0,1.0,1.0,48.525667,4859.0,17.975,101.40,...,3.0,1.0,2885.289954,092321,11.0,6.0,6.0,1375.404960,1.0,1460.394597
27159,92326.0,3.0,9.0,17.0,1.0,2.0,58.973306,4859.0,20.950,108.50,...,3.0,1.0,1368.960567,092326,10.0,2.0,2.0,1365.202694,10.0,1460.394597
27160,92337.0,5.0,9.0,17.0,1.0,2.0,41.494867,3647.0,18.650,101.90,...,3.0,1.0,1368.960567,092337,4.0,6.0,6.0,1365.202694,1.0,1460.394597
27161,92338.0,7.0,9.0,17.0,1.0,2.0,6.439425,11.0,7.775,65.00,...,3.0,1.0,1423.073931,092338,5.0,8.0,8.0,1524.762904,1.0,1460.394597
